In [1]:
from os.path import join, dirname
from dotenv import load_dotenv

# Load settings from environments
dotenv_path = join(dirname('.'), '.env')
load_dotenv(dotenv_path, override=False)

True

In [2]:
from dqa.utils import load_template, load_config
from dqa.expert import Expert

# Load configuration
config = load_config('experts.yml')
template = load_template('prompt_template.txt')

# Init experts
experts = [
    Expert(
        **expert,
        prompt_template=expert.get('prompt_template', template)
    )
    for expert in config['experts']
]

In [3]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("IlyaGusev/saiga_scored", split="train[:10]")  # Replace with your desired dataset name

In [6]:
import json
from dqa.expert import rate_sample

# Example usage
ratings_by_experts = []
for data in dataset:
    sample = json.dumps(data["messages"], ensure_ascii=False)
    ratings_by_expert = rate_sample(sample, experts, max_workers=len(experts))
    ratings_by_experts.append(ratings_by_expert)

ratings_by_experts

Model: google/gemma-2-9b-it - Invalid rating value in response: ::::::::
Model: google/gemma-2-9b-it - Invalid rating value in response: :::::::
Model: google/gemma-2-9b-it - Invalid rating value in response: :::::::
Model: perplexity/llama-3-sonar-small-32k-online - Invalid rating value in response: I have read the sources carefully. Please go ahead
Skipping sample: Size exceeds limit (10593 tokens)
Model: google/gemma-2-9b-it - Invalid rating value in response: 














Skipping sample: Size exceeds limit (7171 tokens)
Model: google/gemma-2-9b-it - Invalid rating value in response: :::::::


[{'gpt-3.5-turbo': 5,
  'anthropic/claude-3-haiku': 5,
  'perplexity/llama-3-sonar-small-32k-online': 4,
  'google/palm-2-chat-bison-32k': 4,
  'google/gemma-2-9b-it': None},
 {'gpt-3.5-turbo': 5,
  'anthropic/claude-3-haiku': 4,
  'perplexity/llama-3-sonar-small-32k-online': 4,
  'google/palm-2-chat-bison-32k': 4,
  'google/gemma-2-9b-it': None},
 {'gpt-3.5-turbo': 5,
  'anthropic/claude-3-haiku': 4,
  'perplexity/llama-3-sonar-small-32k-online': 4,
  'google/palm-2-chat-bison-32k': 4,
  'google/gemma-2-9b-it': None},
 {'gpt-3.5-turbo': 5,
  'anthropic/claude-3-haiku': 5,
  'perplexity/llama-3-sonar-small-32k-online': 4,
  'google/palm-2-chat-bison-32k': 4,
  'google/gemma-2-9b-it': 4},
 {'gpt-3.5-turbo': 5,
  'anthropic/claude-3-haiku': 5,
  'perplexity/llama-3-sonar-small-32k-online': None,
  'google/palm-2-chat-bison-32k': 4,
  'google/gemma-2-9b-it': 4},
 None,
 {'gpt-3.5-turbo': 5,
  'anthropic/claude-3-haiku': 5,
  'perplexity/llama-3-sonar-small-32k-online': 4,
  'google/palm-2

In [7]:
from dqa.helpers import classify_rating

# Process each element concurrently using a pool of experts and take a quorum vote
quorum = (len(experts) // 2) + 1  # (50%+1) calculate the minimum number of experts to get a majority vote

# Calculate the average rating and classify it as "bad" or "good" for each sample
results = classify_rating(ratings_by_experts, quorum)

results

Sample 1: AVG Rating: 4.5, Class: Good
Ratings: [5, 5, 4, 4]
Majority Vote Achieved: True

Sample 2: AVG Rating: 4.25, Class: Good
Ratings: [5, 4, 4, 4]
Majority Vote Achieved: True

Sample 3: AVG Rating: 4.25, Class: Good
Ratings: [5, 4, 4, 4]
Majority Vote Achieved: True

Sample 4: AVG Rating: 4.4, Class: Good
Ratings: [5, 5, 4, 4, 4]
Majority Vote Achieved: True

Sample 5: AVG Rating: 4.5, Class: Good
Ratings: [5, 5, 4, 4]
Majority Vote Achieved: True

Sample 7: AVG Rating: 4.4, Class: Good
Ratings: [5, 5, 4, 4, 4]
Majority Vote Achieved: True

Sample 8: AVG Rating: 4.75, Class: Good
Ratings: [5, 5, 4, 5]
Majority Vote Achieved: True

Sample 10: AVG Rating: 4.5, Class: Good
Ratings: [5, 5, 4, 4]
Majority Vote Achieved: True



[{'sample_index': 1,
  'average_rating': 4.5,
  'classification': 'Good',
  'majority_vote': True},
 {'sample_index': 2,
  'average_rating': 4.25,
  'classification': 'Good',
  'majority_vote': True},
 {'sample_index': 3,
  'average_rating': 4.25,
  'classification': 'Good',
  'majority_vote': True},
 {'sample_index': 4,
  'average_rating': 4.4,
  'classification': 'Good',
  'majority_vote': True},
 {'sample_index': 5,
  'average_rating': 4.5,
  'classification': 'Good',
  'majority_vote': True},
 {'sample_index': 7,
  'average_rating': 4.4,
  'classification': 'Good',
  'majority_vote': True},
 {'sample_index': 8,
  'average_rating': 4.75,
  'classification': 'Good',
  'majority_vote': True},
 {'sample_index': 10,
  'average_rating': 4.5,
  'classification': 'Good',
  'majority_vote': True}]